In [1]:
import os
import datetime

import IPython
import IPython.display
import pandas as pd
import numpy as np
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_excel("preparedData.xlsx")

In [3]:
df = df.T.drop_duplicates().T

In [4]:
df.set_index('date', inplace=True)

In [5]:
df = df.astype(float)

In [6]:
df.reset_index(inplace=True)

In [7]:
df.drop(df.tail(1).index, inplace = True)

In [8]:
df.isnull().values.any()

False

In [9]:
df.isnull().sum().sum()

0

In [10]:
df[df.isna().any(axis=1)]

,date,new_cases_smoothed,reproduction_rate,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,...,Hand sanitizer: (Ireland),Face mask: (Ireland),covid test dublin: (Ireland),covid test centre: (Ireland),hse covid vaccine: (Ireland),hse vaccine portal: (Ireland),hse portal vaccine: (Ireland),pcr test hse: (Ireland),hse covid test: (Ireland),hse vaccine registration: (Ireland)


In [11]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
new_cases_smoothed,701.0,1.760895e+03,3.401838e+03,6.571,2.864290e+02,5.355710e+02,1.492857e+03,2.375200e+04
reproduction_rate,701.0,1.077603e+00,3.467930e-01,0.000,8.800000e-01,1.020000e+00,1.250000e+00,2.840000e+00
new_tests,701.0,1.591973e+04,9.184721e+03,0.000,9.786000e+03,1.537700e+04,2.066200e+04,4.899100e+04
positive_rate,701.0,7.699486e-02,9.589545e-02,0.000,2.280000e-02,4.020000e-02,8.940000e-02,5.523000e-01
tests_per_case,701.0,5.035663e+01,9.392986e+01,0.000,1.040000e+01,2.240000e+01,4.290000e+01,6.830000e+02
total_vaccinations,701.0,2.890186e+06,3.632310e+06,0.000,0.000000e+00,4.567030e+05,6.685455e+06,1.050748e+07
people_vaccinated,701.0,1.452148e+06,1.682086e+06,0.000,0.000000e+00,3.042430e+05,3.642154e+06,4.038305e+06
people_fully_vaccinated,701.0,1.256374e+06,1.619298e+06,0.000,0.000000e+00,1.524770e+05,3.276473e+06,3.939271e+06
total_boosters,701.0,2.589367e+05,7.021485e+05,0.000,0.000000e+00,0.000000e+00,0.000000e+00,2.768440e+06
new_vaccinations,701.0,1.493116e+04,2.001811e+04,0.000,0.000000e+00,6.950000e+03,2.384700e+04,1.351550e+05


In [12]:
X = df[['reproduction_rate', 'new_tests',
       'positive_rate', 'tests_per_case', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'stringency_index', 'population',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'life_expectancy', 'human_development_index', 'covid: (Ireland)',
       'COVID-19 testing: (Ireland)', 
       'Health Service Executive: (Ireland)', 
       'book covid test: (Ireland)_x', 'how many covid cases today: (Ireland)',
       'pcr covid test: (Ireland)', 'close contact covid: (Ireland)',
       'book a covid test: (Ireland)', 'vaccination centre: (Ireland)',
       'pharmacy near me: (Ireland)',
       'Treatment and management of COVID-19: (Ireland)',
       'Hand sanitizer: (Ireland)', 'Face mask: (Ireland)',
       'covid test dublin: (Ireland)',
       'covid test centre: (Ireland)', 'hse covid vaccine: (Ireland)',
       'hse vaccine portal: (Ireland)', 'hse portal vaccine: (Ireland)',
       'pcr test hse: (Ireland)', 'hse covid test: (Ireland)',
       'hse vaccine registration: (Ireland)']]

In [13]:
X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
reproduction_rate,701.0,1.077603e+00,3.467930e-01,0.000,8.800000e-01,1.020000e+00,1.250000e+00,2.840000e+00
new_tests,701.0,1.591973e+04,9.184721e+03,0.000,9.786000e+03,1.537700e+04,2.066200e+04,4.899100e+04
positive_rate,701.0,7.699486e-02,9.589545e-02,0.000,2.280000e-02,4.020000e-02,8.940000e-02,5.523000e-01
tests_per_case,701.0,5.035663e+01,9.392986e+01,0.000,1.040000e+01,2.240000e+01,4.290000e+01,6.830000e+02
total_vaccinations,701.0,2.890186e+06,3.632310e+06,0.000,0.000000e+00,4.567030e+05,6.685455e+06,1.050748e+07
people_vaccinated,701.0,1.452148e+06,1.682086e+06,0.000,0.000000e+00,3.042430e+05,3.642154e+06,4.038305e+06
people_fully_vaccinated,701.0,1.256374e+06,1.619298e+06,0.000,0.000000e+00,1.524770e+05,3.276473e+06,3.939271e+06
total_boosters,701.0,2.589367e+05,7.021485e+05,0.000,0.000000e+00,0.000000e+00,0.000000e+00,2.768440e+06
new_vaccinations,701.0,1.493116e+04,2.001811e+04,0.000,0.000000e+00,6.950000e+03,2.384700e+04,1.351550e+05
stringency_index,701.0,6.218435e+01,1.966235e+01,0.000,4.444000e+01,5.556000e+01,8.148000e+01,9.074000e+01


In [14]:
X.dtypes

reproduction_rate                                  float64
new_tests                                          float64
positive_rate                                      float64
tests_per_case                                     float64
total_vaccinations                                 float64
people_vaccinated                                  float64
people_fully_vaccinated                            float64
total_boosters                                     float64
new_vaccinations                                   float64
stringency_index                                   float64
population                                         float64
population_density                                 float64
median_age                                         float64
aged_65_older                                      float64
aged_70_older                                      float64
gdp_per_capita                                     float64
extreme_poverty                                    float

In [15]:
vif_data = pd.DataFrame()

In [16]:
vif_data['feature'] = X.columns

In [17]:
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

/Users/simonmclain/miniforge3/envs/tf24/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [18]:
vif_data

,feature,VIF
0,reproduction_rate,5.064085
1,new_tests,5.632915
2,positive_rate,8.452559
3,tests_per_case,4.076460
4,total_vaccinations,401369.312803
5,people_vaccinated,83282.531339
6,people_fully_vaccinated,72253.264367
7,total_boosters,15281.935122
8,new_vaccinations,4.664811
9,stringency_index,9.350332


In [19]:
vif_data_new = vif_data[vif_data['VIF']<5]

In [20]:
vif_data_new = vif_data_new[vif_data_new['VIF']>0.1]

In [21]:
vif_data_new

,feature,VIF
3,tests_per_case,4.076460
8,new_vaccinations,4.664811
33,Treatment and management of COVID-19: (Ireland),2.124113
34,Hand sanitizer: (Ireland),4.315733
35,Face mask: (Ireland),4.344853


In [22]:
vif_cols = vif_data_new.feature.to_list()

In [23]:
print(vif_cols)

['tests_per_case', 'new_vaccinations', 'Treatment and management of COVID-19: (Ireland)', 'Hand sanitizer: (Ireland)', 'Face mask: (Ireland)']


In [24]:
add_cols = ['reproduction_rate','date','new_cases_smoothed']

In [25]:
vif_cols.extend(add_cols)

In [26]:
vif_cols

['tests_per_case',
 'new_vaccinations',
 'Treatment and management of COVID-19: (Ireland)',
 'Hand sanitizer: (Ireland)',
 'Face mask: (Ireland)',
 'reproduction_rate',
 'date',
 'new_cases_smoothed']

In [27]:
df_final = df.loc[:,['tests_per_case',
 'new_vaccinations',
 'Treatment and management of COVID-19: (Ireland)',
 'Hand sanitizer: (Ireland)',
 'Face mask: (Ireland)',
 'reproduction_rate',
 'date',
 'new_cases_smoothed']]

In [28]:
df_final.head()

,tests_per_case,new_vaccinations,Treatment and management of COVID-19: (Ireland),Hand sanitizer: (Ireland),Face mask: (Ireland),reproduction_rate,date,new_cases_smoothed
0,0.0,0.0,1.0,68.0,18.0,2.51,2020-03-15,15.714
1,0.0,0.0,2.0,38.0,18.0,2.59,2020-03-16,21.143
2,0.0,0.0,2.0,38.0,18.0,2.70,2020-03-17,27.000
3,0.0,0.0,2.0,38.0,18.0,2.78,2020-03-18,35.571
4,0.0,0.0,2.0,38.0,18.0,2.84,2020-03-19,73.429


In [29]:
df_final.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
tests_per_case,701.0,50.356633,93.929861,0.000,10.400,22.400,42.900,683.00
new_vaccinations,701.0,14931.159772,20018.105179,0.000,0.000,6950.000,23847.000,135155.00
Treatment and management of COVID-19: (Ireland),701.0,0.271041,0.466755,0.000,0.000,0.000,1.000,2.00
Hand sanitizer: (Ireland),701.0,5.069900,6.083182,0.000,2.000,2.000,7.000,68.00
Face mask: (Ireland),701.0,16.352354,17.478884,3.000,6.000,9.000,21.000,100.00
reproduction_rate,701.0,1.077603,0.346793,0.000,0.880,1.020,1.250,2.84
new_cases_smoothed,701.0,1760.894639,3401.837880,6.571,286.429,535.571,1492.857,23752.00


In [30]:
df_final.dtypes

tests_per_case                                            float64
new_vaccinations                                          float64
Treatment and management of COVID-19: (Ireland)           float64
Hand sanitizer: (Ireland)                                 float64
Face mask: (Ireland)                                      float64
reproduction_rate                                         float64
date                                               datetime64[ns]
new_cases_smoothed                                        float64
dtype: object

In [31]:
df_final.set_index('date',inplace=True)

In [32]:
df_final.head()

,tests_per_case,new_vaccinations,Treatment and management of COVID-19: (Ireland),Hand sanitizer: (Ireland),Face mask: (Ireland),reproduction_rate,new_cases_smoothed
date,,,,,,,
2020-03-15,0.0,0.0,1.0,68.0,18.0,2.51,15.714
2020-03-16,0.0,0.0,2.0,38.0,18.0,2.59,21.143
2020-03-17,0.0,0.0,2.0,38.0,18.0,2.70,27.000
2020-03-18,0.0,0.0,2.0,38.0,18.0,2.78,35.571
2020-03-19,0.0,0.0,2.0,38.0,18.0,2.84,73.429


In [33]:
df_final.to_excel("preparedAndCleanedData.xlsx")

In [34]:
# https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/